In [ ]:
#################################################################################
# A set of helper utilities for Bitbucket, functionality supported
# The implementation uses standard rest APIs since the python bitbucket libraries 
# dont work well with cloud version.
#
# Features supported:
# 1. List all repos in a workspace + project
# 2. Update default reviewers (this is idempotent and can be called multiple times)
# 3. Setup branch restricitions (not idempotent and should be called ones, will fail on repeat)
#
# Getting started
# 1. Setup a bitbucket application password
# 2. Setup the environment variables within your pythong notebook, for example run below (do not enter quotes)
# ```
# %env BITBUCKET_USER=<your username>
# %env BITBUCKET_PASS=<application password>
# %env BITBUCKET_URL=https://api.bitbucket.org/
# %env BITBUCKET_WORKSPACE=inspiredbytech
# %env PROJECT_KEY=ABC
# %env USER_ID={xxxx-xxxx-xxxx-xxx-xx}
# ```
#################################################################################

from atlassian.bitbucket.cloud import Cloud
import os, json
import requests
import pandas as pd


username = os.getenv('BITBUCKET_USER', None)
password = os.getenv('BITBUCKET_PASS', None)
url = os.getenv('BITBUCKET_URL', None)
workspace = os.getenv('BITBUCKET_WORKSPACE', None)
project_key = os.getenv('PROJECT_KEY', None)
user_id = os.getenv('USER_ID', None)

In [ ]:
headers = {
    'Content-Type': 'application/json',
}
params = (
    ('key1', 'mykeyhere'),
)
url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/?q=project.key="{project_key}"&limit=1000&pagelen=100'


def get_repo(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 


def get_permissions(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 

In [ ]:
def get_commits(repo_name):
    url =  f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/commits/'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    #print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 
    return rsp.json()

In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
repos_request = requests.get(url,  headers=headers,auth=(username, password))

repos_json = repos_request.json()
df_repos = pd.DataFrame.from_dict(repos_json['values'])

df_report = pd.DataFrame(columns=['hash','repository.name','date', 'author.user.nickname', 'summary.raw'])
df_report.set_index('hash')

"""commits_json = get_commits('infra-compliance')
df_nested_list = pd.json_normalize(commits_json, record_path =['values'])
print(df_nested_list.size)
print(df_nested_list.columns)"""


In [ ]:

import datetime
for repo in df_repos['name']:
    commits_json = get_commits(repo)
    df = pd.DataFrame.from_dict(commits_json['values'])
    df_nested_list = pd.json_normalize(commits_json, record_path =['values'])
    #print(json.dumps(commits_json, sort_keys=True, indent=4))
    #print(df)
    #print(df_nested_list.columns)
    print(repo)
    
    if df_nested_list.size > 0 and 'author.user.nickname' in df_nested_list.columns:
        df_report = pd.concat([df_report, df_nested_list[['hash', 'repository.name','date', 'author.user.nickname', 'summary.raw']]])
print(df_report)

df_report['date_only']=pd.to_datetime(df_report['date'], utc=False).dt.date


In [ ]:
print((datetime.datetime.now() - datetime.timedelta(30)).date())
df_report1 =df_report[df_report['date_only'] > ((datetime.datetime.now() - datetime.timedelta(30)).date())]

df_report1['count'] = 1

#df_groups = df_report.groupby(['date_only', 'author.user.nickname']).size()
df_groups2 = df_report1.pivot_table('count', ['date_only'], 'author.user.nickname')
print(df_groups2.fillna(0))

In [ ]:
df_groups2.plot.area(figsize= (15,15))

In [ ]:
print(df_groups)